In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [6]:
# note scale = 2, or 3 makes it more precise, but too big.
scales = [0]
selected_columns = ['x', 'y']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [7]:
# Verifier/ data consumer side:
from zkstats.computation import computation_to_model, State

def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x, y = data[0], data[1]
    return s.covariance(x, y)

error = 0.01
_, verifier_model = computation_to_model(computation, error)


verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:166: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [8]:
# prover calculates settings, send to verifier
_, prover_model = computation_to_model(computation, error)
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

==== Generate & Calibrate Setting ====
scale:  [0]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":0,"param_scale":0,"scale_rebase_multiplier":10,"lookup_range":[-448,6246],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":13120,"total_assignments":825,"total_const_size":8,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,0],"model_input_scales":[0,0],"module_sizes":{"kzg":[],"poseidon":[13120,[2]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [9]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 3.4178340435028076 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 3215.0
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[14955570959218682635, 4667139652385906200, 12836539004462631467, 1774684518626433649], [4224417983558473805, 851357164555783563, 5363851773531956453, 1448631618362554917], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [5743885005642251665, 3430503978676436355, 7149667244725939006, 2902673458086333540]]], 'proof': '244e8813d8d36db7f4a09827891ae218e6ea378c905a209177d5b103ca7242903029877927960516e93221211bd008454799df5dbf923e49755f424f500348270333819eaac3f03b25eeeb6d533769036712641df0ebdf63519c3be5eac5a49605c6e3b0ece32c87dd7b5ce02accc0015e4e7181339d6cc4ad5f51325ef6143806e8d8578347e5897ca6ab7926f0ed76a727a7cad81553b7972da369af3ef23826e6d2cc5179706d652ddbc46f3a25797129860d6635181cb6e9d6653326b8900f7c83818bf2782e32b816355a3bdfb77fcdf3c1428a5f17ff1c8edb601828ae169f3a239348a3ccde70382d57d68702634ea9bd1d27d3fe05dc41e442b563b9139768f9f6a2e6e6147b2a49da23ac858e8f1fea27ab7a1a4731f6ef44b5c73219fbc41fb5dc496c1327fdd5f50dcb14d14669d403

In [11]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: [3215.0]
